# Import

In [1]:
import re
import pandas        as pd
import numpy         as np

# Load DataSet

In [2]:
df = pd.read_csv('../dataset/list_auto_cleaning.csv', encoding='utf-8', sep=';')
df_2 = pd.read_csv('../dataset/list_auto_2023-10-30.csv', encoding='utf-8', sep=';')


In [3]:
df_2.head()

,id_ads,manufacturer_by,model,version,km,year,price,seller,endereco,scrapy_datetime
0,7fd72920-850d-43ba-a452-cbae1a6398da,land rover,defender,110 SW SE,81000,08-2015,38000,AUTOMOVILES MARTIN,Contáctanos en: • ES-14440 VILLANUEVA DE CÓRDOBA,2023-10-30 12:35:47
1,aeab142d-acdb-4b57-b4e4-f4a8d305af30,land rover,defender,130 D300 X-Dynamic HSE Luftfederung StandHZG AHK,15,new,109900,AUTOMOVILES MARTIN,Contáctanos en: • ES-14440 VILLANUEVA DE CÓRDOBA,2023-10-30 12:35:47
2,7484ee53-ebc5-4510-8bf9-032e657837cf,land rover,defender,110 SW E,78000,08-2012,39500,AUTOMOVILES MARTIN,Contáctanos en: • ES-14440 VILLANUEVA DE CÓRDOBA,2023-10-30 12:35:47
3,9b6915b4-b744-4bd3-bd73-9de4141e42fa,land rover,defender,2.4 Turbo - D E,101706,09-2009,34995,AUTOMOVILES MARTIN,Contáctanos en: • ES-14440 VILLANUEVA DE CÓRDOBA,2023-10-30 12:35:47
4,c29521a8-666e-4c6e-822a-3ae66def0890,land rover,defender,110 SW E,65000,08-2013,41000,AUTOMOVILES MARTIN,Contáctanos en: • ES-14440 VILLANUEVA DE CÓRDOBA,2023-10-30 12:35:47


# Using REGEX langague to clean and capture data-info

In [5]:
df_clean = df_2.copy()

In [6]:
# "Extracting the postal code from the vehicle's country of origin. 
# Applying Regex for data cleaning."
df_clean['zip_code'] = df_clean['endereco'].apply(lambda x: int(re.search('\d+', x).group(0) if pd.notnull( x ) else x ))

In [7]:
# Model
df_clean['model'] = 'DEFENDER' + ' ' + df_clean['version'].apply(lambda x: ' '.join(re.findall(r'\b(88|90|110|130)\b', x)) if re.findall(r'\d+', x) else None)

In [8]:
# Version Type
df_clean['engine'] = df_clean['version'].apply(lambda x: ' '.join(re.findall(r'[A-Za-z]+\d+', x)) if re.findall(r'\d+', x) else None)
df_clean['engine_version'] = df_clean['version'].apply(lambda x: ' '.join(re.findall(r'(AWD|SW|SE|HSE|MHEV|HT)', x)) if re.findall(r'\d+', x) else None)
df_clean['cc_engine'] = df_clean['version'].apply(lambda x: ' '.join(re.findall(r'\b\d\.\d', x)) if re.findall(r'\d+', x) else None)

# Replacing model version values.
df_clean['version'] = df_clean.apply(lambda row: row['engine'] + ' ' + row['engine_version'] + ' ' + row['cc_engine'] if row['engine'] is not None and row['engine_version'] is not None and row['cc_engine'] is not None else None, axis=1)


In [9]:
#Country
df_clean['country'] = df_clean['endereco'].apply(lambda x: str(re.search(r"([A-Z]{2})", x).group(0) if pd.notnull( x ) else x ))

In [10]:
# DataTime cleaning

df_clean['year'] = df_clean['year'].str.replace('-', '/')


for i in range(len(df_clean)):
    if df_clean['year'][i] == 'new' or df_clean['year'][i] == 'unknown' or df_clean['year'][i] == 'None':
        df_clean.at[i, 'year'] = '01/2023'


df_clean['year'] = pd.to_datetime(df_clean['year'], format='%m/%Y')

df_clean['year'] = df_clean['year'].dt.year


## DataFrame Final

In [11]:
df_final = pd.DataFrame(df_clean[['id_ads', 'manufacturer_by', 'model', 'version', 'km', 'year',
                                  'price', 'seller', 'endereco', 'scrapy_datetime', 'zip_code', 'country']])

In [13]:
df_final.head()

,id_ads,manufacturer_by,model,version,km,year,price,seller,endereco,scrapy_datetime,zip_code,country
0,7fd72920-850d-43ba-a452-cbae1a6398da,land rover,DEFENDER 110,SW SE,81000,2015,38000,AUTOMOVILES MARTIN,Contáctanos en: • ES-14440 VILLANUEVA DE CÓRDOBA,2023-10-30 12:35:47,14440,ES
1,aeab142d-acdb-4b57-b4e4-f4a8d305af30,land rover,DEFENDER 130,D300 HSE,15,2023,109900,AUTOMOVILES MARTIN,Contáctanos en: • ES-14440 VILLANUEVA DE CÓRDOBA,2023-10-30 12:35:47,14440,ES
2,7484ee53-ebc5-4510-8bf9-032e657837cf,land rover,DEFENDER 110,SW,78000,2012,39500,AUTOMOVILES MARTIN,Contáctanos en: • ES-14440 VILLANUEVA DE CÓRDOBA,2023-10-30 12:35:47,14440,ES
3,9b6915b4-b744-4bd3-bd73-9de4141e42fa,land rover,DEFENDER,2.4,101706,2009,34995,AUTOMOVILES MARTIN,Contáctanos en: • ES-14440 VILLANUEVA DE CÓRDOBA,2023-10-30 12:35:47,14440,ES
4,c29521a8-666e-4c6e-822a-3ae66def0890,land rover,DEFENDER 110,SW,65000,2013,41000,AUTOMOVILES MARTIN,Contáctanos en: • ES-14440 VILLANUEVA DE CÓRDOBA,2023-10-30 12:35:47,14440,ES


In [12]:
df_final.dtypes

id_ads             object
manufacturer_by    object
model              object
version            object
km                 object
year                int32
price               int64
seller             object
endereco           object
scrapy_datetime    object
zip_code            int64
country            object
dtype: object

## Save the final dataset


In [165]:
rows = df_final.shape[0]
print(rows)
df_final.to_csv('../dataset/df_final.csv', encoding='utf-8', sep=';')

399
